<a href="https://colab.research.google.com/github/GabrielBG0/Fakenews-Recognition/blob/main/BART_%26_20NG_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/CSCfi/machine-learning-scripts/blob/master/examples/pytorch_20ng_bert.py


In [1]:
# Deleting default sample data folder
!rm -rf /content/sample_data

In [2]:
# Install aditional librarys
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 21.2MB/s 
     |████████████████████████████████| 3.3MB 53.7MB/s 
     |████████████████████████████████| 901kB 41.5MB/s 
     |████████████████████████████████| 204kB 19.7MB/s 
     |████████████████████████████████| 245kB 46.1MB/s 
     |████████████████████████████████| 112kB 56.0MB/s 


In [3]:
# Get dataset from 20 News Group
!rm -rf /content/20_newsgroups/
!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
!tar -xf news20.tar.gz

--2021-04-20 23:55:26--  http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17329808 (17M) [application/x-gzip]
Saving to: ‘news20.tar.gz’

news20.tar.gz       100%[===================>]  16.53M  2.47MB/s    in 9.4s    

2021-04-20 23:55:35 (1.76 MB/s) - ‘news20.tar.gz’ saved [17329808/17329808]



In [4]:
import torch
from torch.utils.data import (TensorDataset, DataLoader,
                              RandomSampler, SequentialSampler)

from transformers import BertTokenizer, BertConfig
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from distutils.version import LooseVersion as LV

from sklearn.model_selection import train_test_split

import io, sys, os, datetime

import numpy as np

In [5]:
# Checks if there is a gpu available
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
if 'DATADIR' in os.environ:
    DATADIR = os.environ['DATADIR']
else:
    DATADIR = "/content/"

TEXT_DATA_DIR = os.path.join(DATADIR, "20_newsgroup")

In [8]:
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


In [9]:
# Split the data into a training set and a test set using
# scikit-learn's train_test_split().

TEST_SET = 4000

(sentences_train, sentences_test,
 labels_train, labels_test) = train_test_split(texts, labels,
                                               test_size=TEST_SET,
                                               shuffle=True, random_state=42)

print('Length of training texts:', len(sentences_train))
print('Length of training labels:', len(labels_train))
print('Length of test texts:', len(sentences_test))
print('Length of test labels:', len(labels_test))

Length of training texts: 15997
Length of training labels: 15997
Length of test texts: 4000
Length of test labels: 4000


In [10]:
# The token [CLS] is a special token required by BERT at the beginning
# of the sentence.

sentences_train = ["[CLS] " + s for s in sentences_train]
sentences_test = ["[CLS] " + s for s in sentences_test]

print ("The first training sentence:")
print(sentences_train[0], 'LABEL:', labels_train[0])

The first training sentence:
[CLS] 

In article <C6BFLB.KEM@cs.columbia.edu> ethan@cs.columbia.edu (Ethan Solomita) writes:
>
>	Three q's:
>
>1) is it reliable?

I use it all day every day (maintaining our ftp site and answering mail
via support@qdeck.com), and I can honestly say that in the last few
months I've never had my machine go down due to any sort of tcpip network
manager instability. (Of course, I've crashed my machine quite a few times
on purpose, during beta testing and that sort of thing, but the tcpip
portion is quite stable...)

However, keep in mind that DVX and the network managers are only going
to be as stable as the software they sit on top of (so if your underlying
network kernel is flakey, you can't expect DVX to be terribly stable...)

>2) how does it send the information from a MS Windows app over
>the X11 protocol? Does it just draw everything as graphics into
>one window, or does it use multiple windows and essentially work
>more cleverly?

It just goes as a w

In [11]:
# Next we specify the pre-trained BERT model we are going to use. The
# model `"bert-base-uncased"` is the lowercased "base" model
# (12-layer, 768-hidden, 12-heads, 110M parameters).
#
# We load the used vocabulary from the BERT model, and use the BERT
# tokenizer to convert the sentences into tokens that match the data
# the BERT model was trained on.

print('Initializing BertTokenizer')

BERTMODEL='bert-base-uncased'
CACHE_DIR=os.path.join(DATADIR, 'transformers-cache')

tokenizer = BertTokenizer.from_pretrained(BERTMODEL, cache_dir=CACHE_DIR,
                                          do_lower_case=True)

tokenized_train = [tokenizer.tokenize(s) for s in sentences_train]
tokenized_test  = [tokenizer.tokenize(s) for s in sentences_test]

print ("The full tokenized first training sentence:")
print (tokenized_train[0])

Initializing BertTokenizer



The full tokenized first training sentence:
['[CLS]', 'in', 'article', '<', 'c', '##6', '##bf', '##lb', '.', 'ke', '##m', '@', 'cs', '.', 'columbia', '.', 'ed', '##u', '>', 'ethan', '@', 'cs', '.', 'columbia', '.', 'ed', '##u', '(', 'ethan', 'solo', '##mit', '##a', ')', 'writes', ':', '>', '>', 'three', 'q', "'", 's', ':', '>', '>', '1', ')', 'is', 'it', 'reliable', '?', 'i', 'use', 'it', 'all', 'day', 'every', 'day', '(', 'maintaining', 'our', 'ft', '##p', 'site', 'and', 'answering', 'mail', 'via', 'support', '@', 'q', '##deck', '.', 'com', ')', ',', 'and', 'i', 'can', 'honestly', 'say', 'that', 'in', 'the', 'last', 'few', 'months', 'i', "'", 've', 'never', 'had', 'my', 'machine', 'go', 'down', 'due', 'to', 'any', 'sort', 'of', 'tc', '##pi', '##p', 'network', 'manager', 'instability', '.', '(', 'of', 'course', ',', 'i', "'", 've', 'crashed', 'my', 'machine', 'quite', 'a', 'few', 'times', 'on', 'purpose', ',', 'during', 'beta', 'testing', 'and', 'that', 'sort', 'of', 'thing', ',', 'bu

In [12]:
# Now we set the maximum sequence lengths for our training and test
# sentences as `MAX_LEN_TRAIN` and `MAX_LEN_TEST`. The maximum length
# supported by the used BERT model is 512.
#
# The token `[SEP]` is another special token required by BERT at the
# end of the sentence.

MAX_LEN_TRAIN, MAX_LEN_TEST = 128, 512

tokenized_train = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized_train]
tokenized_test  = [t[:(MAX_LEN_TEST-1)]+['SEP'] for t in tokenized_test]

print ("The truncated tokenized first training sentence:")
print (tokenized_train[0])

The truncated tokenized first training sentence:
['[CLS]', 'in', 'article', '<', 'c', '##6', '##bf', '##lb', '.', 'ke', '##m', '@', 'cs', '.', 'columbia', '.', 'ed', '##u', '>', 'ethan', '@', 'cs', '.', 'columbia', '.', 'ed', '##u', '(', 'ethan', 'solo', '##mit', '##a', ')', 'writes', ':', '>', '>', 'three', 'q', "'", 's', ':', '>', '>', '1', ')', 'is', 'it', 'reliable', '?', 'i', 'use', 'it', 'all', 'day', 'every', 'day', '(', 'maintaining', 'our', 'ft', '##p', 'site', 'and', 'answering', 'mail', 'via', 'support', '@', 'q', '##deck', '.', 'com', ')', ',', 'and', 'i', 'can', 'honestly', 'say', 'that', 'in', 'the', 'last', 'few', 'months', 'i', "'", 've', 'never', 'had', 'my', 'machine', 'go', 'down', 'due', 'to', 'any', 'sort', 'of', 'tc', '##pi', '##p', 'network', 'manager', 'instability', '.', '(', 'of', 'course', ',', 'i', "'", 've', 'crashed', 'my', 'machine', 'quite', 'a', 'few', 'times', 'on', 'purpose', ',', 'during', 'beta', 'testing', 'SEP']


In [13]:
# Next we use the BERT tokenizer to convert each token into an integer
# index in the BERT vocabulary. We also pad any shorter sequences to
# `MAX_LEN_TRAIN` or `MAX_LEN_TEST` indices with trailing zeros.

ids_train = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_train]
ids_train = np.array([np.pad(i, (0, MAX_LEN_TRAIN-len(i)),
                             mode='constant') for i in ids_train])

ids_test = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_test]
ids_test = np.array([np.pad(i, (0, MAX_LEN_TEST-len(i)),
                            mode='constant') for i in ids_test])

print ("The indices of the first training sentence:")
print (ids_train[0])

The indices of the first training sentence:
[  101  1999  3720  1026  1039  2575 29292 20850  1012 17710  2213  1030
 20116  1012  3996  1012  3968  2226  1028  6066  1030 20116  1012  3996
  1012  3968  2226  1006  6066  3948 22930  2050  1007  7009  1024  1028
  1028  2093  1053  1005  1055  1024  1028  1028  1015  1007  2003  2009
 10539  1029  1045  2224  2009  2035  2154  2296  2154  1006  8498  2256
  3027  2361  2609  1998 10739  5653  3081  2490  1030  1053 26547  1012
  4012  1007  1010  1998  1045  2064  9826  2360  2008  1999  1996  2197
  2261  2706  1045  1005  2310  2196  2018  2026  3698  2175  2091  2349
  2000  2151  4066  1997 22975  8197  2361  2897  3208 18549  1012  1006
  1997  2607  1010  1045  1005  2310  8007  2026  3698  3243  1037  2261
  2335  2006  3800  1010  2076  8247  5604   100]


In [14]:
# BERT also requires *attention masks*, with 1 for each real token in
# the sequences and 0 for the padding:

amasks_train, amasks_test = [], []

for seq in ids_train:
  seq_mask = [float(i>0) for i in seq]
  amasks_train.append(seq_mask)

for seq in ids_test:
  seq_mask = [float(i>0) for i in seq]
  amasks_test.append(seq_mask)

In [15]:
# We use again scikit-learn's train_test_split to use 10% of our
# training data as a validation set, and then convert all data into
# torch.tensors.

(train_inputs, validation_inputs,
 train_labels, validation_labels) = train_test_split(ids_train, labels_train,
                                                     random_state=42,
                                                     test_size=0.1)
(train_masks, validation_masks,
 _, _) = train_test_split(amasks_train, ids_train,
                          random_state=42, test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks  = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks  = torch.tensor(validation_masks)
test_inputs = torch.tensor(ids_test)
test_labels = torch.tensor(labels_test)
test_masks  = torch.tensor(amasks_test)

In [16]:
# Next we create PyTorch DataLoaders for all data sets.
#
# For fine-tuning BERT on a specific task, the authors recommend a
# batch size of 16 or 32.

BATCH_SIZE = 32

print('Train: ', end="")
train_data = TensorDataset(train_inputs, train_masks,
                           train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler,
                              batch_size=BATCH_SIZE)
print(len(train_data), 'messages')

print('Validation: ', end="")
validation_data = TensorDataset(validation_inputs, validation_masks,
                                validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,
                                   sampler=validation_sampler,
                                   batch_size=BATCH_SIZE)
print(len(validation_data), 'messages')

print('Test: ', end="")
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler,
                             batch_size=BATCH_SIZE)
print(len(test_data), 'messages')

Train: 14397 messages
Validation: 1600 messages
Test: 4000 messages


In [17]:
# ## BERT model initialization
#
# We now load a pretrained BERT model with a single linear
# classification layer added on top.

print('Initializing BertForSequenceClassification')

model = BertForSequenceClassification.from_pretrained(BERTMODEL,
                                                      cache_dir=CACHE_DIR,
                                                      num_labels=20)
model.cuda()

Initializing BertForSequenceClassification


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
# We set the remaining hyperparameters needed for fine-tuning the
# pretrained model:
#   * EPOCHS: the number of training epochs in fine-tuning
#     (recommended values between 2 and 4)
#   * WEIGHT_DECAY: weight decay for the Adam optimizer
#   * LR: learning rate for the Adam optimizer (2e-5 to 5e-5 recommended)
#   * WARMUP_STEPS: number of warmup steps to (linearly) reach the set
#     learning rate
#
# We also need to grab the training parameters from the pretrained model.

EPOCHS = 16
WEIGHT_DECAY = 0.01
LR = 2e-5
WARMUP_STEPS =int(0.2*len(train_dataloader))

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=LR, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=WARMUP_STEPS,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [19]:
# ## Learning
#
# Let's now define functions to train() and evaluate() the model:

def train(epoch, loss_vector=None, log_interval=200):
  # Set model to training mode
  model.train()

  # Loop over each batch from the training set
  for step, batch in enumerate(train_dataloader):

    # Copy data to GPU if needed
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Zero gradient buffers
    optimizer.zero_grad()

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None,
                    attention_mask=b_input_mask, labels=b_labels)

    loss = outputs[0]
    if loss_vector is not None:
        loss_vector.append(loss.item())

    # Backward pass
    loss.backward()

    # Update weights
    optimizer.step()
    scheduler.step()

    if step % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, step * len(b_input_ids),
                len(train_dataloader.dataset),
                100. * step / len(train_dataloader), loss))

def evaluate(loader):
  model.eval()

  n_correct, n_all = 0, 0

  for batch in loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)

    labels = b_labels.to('cpu').numpy()
    n_correct += np.sum(predictions == labels)
    n_all += len(labels)

  print('Accuracy: [{}/{}] {:.4f}\n'.format(n_correct,
                                            n_all,
                                            n_correct/n_all))

# Now we are ready to train our model using the train()
# function. After each epoch, we evaluate the model using the
# validation set and evaluate().

train_lossv = []
for epoch in range(1, EPOCHS + 1):
    train(epoch, train_lossv)
    print('\nValidation set:')
    evaluate(validation_dataloader)

# ## Inference
#
# For a better measure of the quality of the model, let's see the
# model accuracy for the test messages.

print('Test set:')
evaluate(test_dataloader)

Train Epoch: 1 [0/14397 (0%)]	Loss: 3.013786
Train Epoch: 1 [6400/14397 (44%)]	Loss: 1.511133
Train Epoch: 1 [12800/14397 (89%)]	Loss: 1.355298

Validation set:
Accuracy: [1126/1600] 0.7037

Train Epoch: 2 [0/14397 (0%)]	Loss: 0.666077
Train Epoch: 2 [6400/14397 (44%)]	Loss: 0.775207
Train Epoch: 2 [12800/14397 (89%)]	Loss: 0.714356

Validation set:
Accuracy: [1214/1600] 0.7588

Train Epoch: 3 [0/14397 (0%)]	Loss: 0.716388
Train Epoch: 3 [6400/14397 (44%)]	Loss: 0.499021
Train Epoch: 3 [12800/14397 (89%)]	Loss: 0.343920

Validation set:
Accuracy: [1240/1600] 0.7750

Train Epoch: 4 [0/14397 (0%)]	Loss: 0.362978
Train Epoch: 4 [6400/14397 (44%)]	Loss: 0.503422
Train Epoch: 4 [12800/14397 (89%)]	Loss: 0.425550

Validation set:
Accuracy: [1260/1600] 0.7875

Train Epoch: 5 [0/14397 (0%)]	Loss: 0.134550
Train Epoch: 5 [6400/14397 (44%)]	Loss: 0.246408
Train Epoch: 5 [12800/14397 (89%)]	Loss: 0.477131

Validation set:
Accuracy: [1270/1600] 0.7937

Train Epoch: 6 [0/14397 (0%)]	Loss: 0.392404
